# Lab 7-1: Tips

### Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

### Training and Test Datasets

In [2]:
x_train = torch.FloatTensor([[1, 2, 1],
                             [1, 3, 2],
                             [1, 3, 4],
                             [1, 5, 5],
                             [1, 7, 5],
                             [1, 2, 5],
                             [1, 6, 6],
                             [1, 7, 7]
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])

x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

### Model

In [5]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
    
    def forward(self, x):
        return self.linear(x)
    
model = SoftmaxClassifierModel()

# optimizer
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [6]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        # H(x)
        hypothesis = model(x_train)
        
        # cost
        cost = F.cross_entropy(hypothesis, y_train)
        
        # cost로 F(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

In [13]:
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    predicted_classes = prediction.max(1)[1]
    correct_count = (predicted_classes == y_test).sum().item()
    cost = F.cross_entropy(prediction, y_test)
    
    print('Accuracy: {}% Cost: {:.6f}'.format(
         correct_count / len(y_test) * 100, cost.item()
    ))

In [14]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 1.280268
Epoch    1/20 Cost: 1.007498
Epoch    2/20 Cost: 0.999968
Epoch    3/20 Cost: 0.992704
Epoch    4/20 Cost: 0.985657
Epoch    5/20 Cost: 0.978815
Epoch    6/20 Cost: 0.972169
Epoch    7/20 Cost: 0.965709
Epoch    8/20 Cost: 0.959425
Epoch    9/20 Cost: 0.953312
Epoch   10/20 Cost: 0.947360
Epoch   11/20 Cost: 0.941562
Epoch   12/20 Cost: 0.935913
Epoch   13/20 Cost: 0.930405
Epoch   14/20 Cost: 0.925033
Epoch   15/20 Cost: 0.919792
Epoch   16/20 Cost: 0.914675
Epoch   17/20 Cost: 0.909678
Epoch   18/20 Cost: 0.904796
Epoch   19/20 Cost: 0.900025


In [15]:
test(model, optimizer, x_test, y_test)

Accuracy: 100.0% Cost: 0.390032


### Data Preprocessing

In [16]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])

> - $ x_j = \frac{x_j - \mu_j}{\sigma_j} $
- $\sigma$ : standard deviation
- $\mu$ : 평균값

In [25]:
mu = x_train.mean(dim = 0)
sigma = x_train.std(dim = 0)
norm_x_train = (x_train - mu) / sigma
norm_x_train

tensor([[-1.0674, -0.3758, -0.8398],
        [ 0.7418,  0.2778,  0.5863],
        [ 0.3799,  0.5229,  0.3486],
        [ 1.0132,  1.0948,  1.1409],
        [-1.0674, -1.5197, -1.2360]])

In [26]:
class MultivariateLinearRegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3,1)
        
    def forward(self, x):
        return self.linear(x)
    
model = MultivariateLinearRegressionModel()

optimizer = optim.SGD(model.parameters(), lr=1e-1)

In [27]:
def train(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        # H(x)
        hypothesis = model(x_train)
        
        # cost
        cost = F.mse_loss(hypothesis, y_train)
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))
        
train(model, optimizer, norm_x_train, y_train)

Epoch    0/20 Cost: 29729.949219
Epoch    1/20 Cost: 18889.082031
Epoch    2/20 Cost: 12048.978516
Epoch    3/20 Cost: 7699.844727
Epoch    4/20 Cost: 4924.700195
Epoch    5/20 Cost: 3151.020264
Epoch    6/20 Cost: 2016.562866
Epoch    7/20 Cost: 1290.709229
Epoch    8/20 Cost: 826.216003
Epoch    9/20 Cost: 528.952271
Epoch   10/20 Cost: 338.703308
Epoch   11/20 Cost: 216.940033
Epoch   12/20 Cost: 139.007050
Epoch   13/20 Cost: 89.125130
Epoch   14/20 Cost: 57.196125
Epoch   15/20 Cost: 36.757317
Epoch   16/20 Cost: 23.672049
Epoch   17/20 Cost: 15.293401
Epoch   18/20 Cost: 9.927165
Epoch   19/20 Cost: 6.488902


### Overfitting

> - 많은 양의 학습 데이터
- 간단한 layer
- **Regularization**

In [29]:
def train_with_regularization(model, optimizer, x_train, y_train):
    nb_epochs = 20
    for epoch in range(nb_epochs):
        # H(x)
        prediction = model(x_train)
        
        # cost
        cost = F.mse_loss(prediction, y_train)
        
        # l2 norm
        l2_reg = 0
        for param in model.parameters():
            l2_reg += torch.norm(param)
            
        cost += l2_reg
        
        # cost로 H(x) 개선
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch+1, nb_epochs, cost.item()
        ))
        
model = MultivariateLinearRegressionModel()
optimizer = optim.SGD(model.parameters(), lr=1e-1)
train_with_regularization(model, optimizer, norm_x_train, y_train)

Epoch    1/20 Cost: 29632.869141
Epoch    2/20 Cost: 18897.533203
Epoch    3/20 Cost: 12122.802734
Epoch    4/20 Cost: 7814.004395
Epoch    5/20 Cost: 5064.236816
Epoch    6/20 Cost: 3306.669922
Epoch    7/20 Cost: 2182.488525
Epoch    8/20 Cost: 1463.201782
Epoch    9/20 Cost: 1002.911377
Epoch   10/20 Cost: 708.338318
Epoch   11/20 Cost: 519.812378
Epoch   12/20 Cost: 399.154022
Epoch   13/20 Cost: 321.930054
Epoch   14/20 Cost: 272.503662
Epoch   15/20 Cost: 240.868011
Epoch   16/20 Cost: 220.618408
Epoch   17/20 Cost: 207.656235
Epoch   18/20 Cost: 199.357986
Epoch   19/20 Cost: 194.044785
Epoch   20/20 Cost: 190.642242
